In [1]:
from os import listdir
from os.path import isfile, join
import pickle
import tensorflow.keras.saving as ker_saving

In [2]:
model_base = "../models"

res = []
models = dict()

for f_name in listdir(model_base):
    path = join(model_base, f_name)
    if isfile(path):
        try:
            with open(path, 'rb') as f:
                model = pickle.load(f)
                models[f_name] = model
        except pickle.UnpicklingError:
            model = ker_saving.load_model(path)
            models[f_name] = model

e:\Projects\HSE\HSE_ML_SP2024\Env\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.4.1.post1 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
e:\Projects\HSE\HSE_ML_SP2024\Env\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.4.1.post1 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
e:\Projects\HSE\HSE_ML_SP2024\Env\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 

In [3]:
for name, model in models.items():
    print(name)

CatBoostClassifier.pkl
LogisticRegression.pkl
nn.keras
RandomForestClassifier.pkl


In [6]:
import pandas as pd
base = "../data"
prep = f"{base}/prepared.csv"
df_small = pd.read_csv(prep)

In [7]:
df_small.dropna(subset=['text'], inplace=True)
df_small.sample()

,toxic,text
98054,1.0,unsolicited advice hi pigsonthewing someone si...


In [8]:
X, y = df_small['text'], df_small['toxic']